In [1]:
!pip install -qU langchain\
    langchain-community\
    langchain-core\
    gpt4all\
    langgraph\
    chromadb\
    sentence-transformers\
    langchain-huggingface

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

In [2]:
!pip install pandas

In [3]:
!pip install chromadb

In [4]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 23.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3511083 sha256=915a1e58e933f4c3f3d19d35728513901060a9ca95e8ce6d6ae2d6711239f111
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [5]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [8]:
!pip install faiss-gpu

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.6 MB/s eta 0:00:00:00:0100:01


In [9]:
csv_file_path = "/kaggle/input/amazonreviews/Amazon_Reviews.csv"
qa_data = pd.read_csv(csv_file_path)

loader = CSVLoader(file_path=csv_file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
chunks = text_splitter.split_documents(docs)

embedding_function = HuggingFaceEmbeddings(show_progress=True, multi_process=False)

vector_store = FAISS.from_documents(documents=chunks, embedding=embedding_function)
retriever = vector_store.as_retriever()

template = """You are an assistant for specific knowledge query tasks.
   Use the following pieces of retrieved context to answer the question.
   If you don't know the answer, just say that you don't know.
   Question: {question}
   Context: {context}
   Answer:
   """
prompt = ChatPromptTemplate.from_template(template)


n_gpu_layers = 32  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="/kaggle/input/llama2-7b-chat/gguf/llama2gguf/1/llama-2-7b-chat.Q4_K_M.gguf",
    #model_path="Meta-Llama-3-8B-Instruct.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    # model_path="Meta-Llama-3-8B-Instruct.Q4_K_M.gguf",
    n_ctx=1024,
    f16_kv=True,
    #temperature=0.75,
    #max_tokens=500,
    # top_p=1,
    callback_manager=callback_manager,
    verbose=True,
)

rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()} 
        | prompt
        | llm
        | StrOutputParser()
)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /kaggle/input/llama2-7b-chat/gguf/llama2gguf/1/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

In [12]:
query = "What are the specs of s22 ultra?"
print(rag_chain.invoke(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 52 prefix-match hit, remaining 681 prompt tokens to eval


 The Samsung Galaxy S22's specs are:
* Processor: Snapdragon 8 Gen 1 or Exynos 2200 (depending on the region)
* RAM: 8GB
* Storage: 128GB internal storage
* Camera: 50MP primary camera, 10MP front-facing camera
* Battery: 3700mAh battery
* Other features: Fingerprint sensor, IP68 rating, stereo speakers, and USB Type-C.
I don't know the answer to your question regarding the S22 Ultra as there is no information provided in the context you provided.

llama_perf_context_print:        load time =  107859.87 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   681 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   153 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  168198.73 ms /   834 tokens


 The Samsung Galaxy S22's specs are:
* Processor: Snapdragon 8 Gen 1 or Exynos 2200 (depending on the region)
* RAM: 8GB
* Storage: 128GB internal storage
* Camera: 50MP primary camera, 10MP front-facing camera
* Battery: 3700mAh battery
* Other features: Fingerprint sensor, IP68 rating, stereo speakers, and USB Type-C.
I don't know the answer to your question regarding the S22 Ultra as there is no information provided in the context you provided.


In [13]:
query = "Can you tell me about LLMs?"
print(rag_chain.invoke(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 51 prefix-match hit, remaining 568 prompt tokens to eval


 LLMS (Language Model Systems) are artificial intelligence systems trained on vast amounts of text data to perform various natural language processing tasks. They have become increasingly popular in recent years due to their ability to generate coherent and contextually relevant text.
Some common applications of LLMs include:
1. Language Translation: LLMs can be used to translate text from one language to another, allowing for more accurate and natural-sounding translations.
2. Text Summarization: LLMs can summarize long pieces of text into shorter, more digestible versions while preserving the essential information.
3. Sentiment Analysis: LLMs can analyze text to determine the sentiment or emotional tone behind it, such as detecting whether a piece of text is positive, negative, or neutral.
4. Chatbots and Conversational AI: LLMs can be used to generate responses to user input in chatbots and other conversational interfaces, allowing for more natural-sounding and engaging interactions

llama_perf_context_print:        load time =  107859.87 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   568 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  178816.11 ms /   823 tokens


 LLMS (Language Model Systems) are artificial intelligence systems trained on vast amounts of text data to perform various natural language processing tasks. They have become increasingly popular in recent years due to their ability to generate coherent and contextually relevant text.
Some common applications of LLMs include:
1. Language Translation: LLMs can be used to translate text from one language to another, allowing for more accurate and natural-sounding translations.
2. Text Summarization: LLMs can summarize long pieces of text into shorter, more digestible versions while preserving the essential information.
3. Sentiment Analysis: LLMs can analyze text to determine the sentiment or emotional tone behind it, such as detecting whether a piece of text is positive, negative, or neutral.
4. Chatbots and Conversational AI: LLMs can be used to generate responses to user input in chatbots and other conversational interfaces, allowing for more natural-sounding and engaging interactions

In [ ]:
query = "What fualts can be faced in s22?"
print(rag_chain.invoke(query))

In [14]:
query = "What improvements can be made in s22 for its better selling?"
print(rag_chain.invoke(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 51 prefix-match hit, remaining 639 prompt tokens to eval


 Based on the provided context, it appears that there are several improvements that could be made to improve the selling of the Samsung Galaxy S22:
1. Better battery life: Many reviewers have noted that the battery life of the S22 can be a drawback, particularly if used heavily throughout the day. Improving the battery capacity or implementing more efficient power management features could help address this issue.
2. Enhanced fingerprint sensor functionality: The fingerprint sensor on the S22 has been criticized for its poor accuracy and frequent lockouts. Investing in a better sensor technology or improving the current one's algorithms could help improve user experience.
3. Expandable storage options: Some reviewers have noted that the limited internal storage capacity of the S22 can be a downside, especially for users who want to store more apps, photos, and videos. Providing expandable storage options like a microSD card slot or cloud storage integration could help address this issu

llama_perf_context_print:        load time =  107859.87 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   639 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  198153.08 ms /   894 tokens


 Based on the provided context, it appears that there are several improvements that could be made to improve the selling of the Samsung Galaxy S22:
1. Better battery life: Many reviewers have noted that the battery life of the S22 can be a drawback, particularly if used heavily throughout the day. Improving the battery capacity or implementing more efficient power management features could help address this issue.
2. Enhanced fingerprint sensor functionality: The fingerprint sensor on the S22 has been criticized for its poor accuracy and frequent lockouts. Investing in a better sensor technology or improving the current one's algorithms could help improve user experience.
3. Expandable storage options: Some reviewers have noted that the limited internal storage capacity of the S22 can be a downside, especially for users who want to store more apps, photos, and videos. Providing expandable storage options like a microSD card slot or cloud storage integration could help address this issu

In [15]:
query = "What is the most major fault in s22?"
print(rag_chain.invoke(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 52 prefix-match hit, remaining 601 prompt tokens to eval


 I don't know the answer to your question as none of the provided context contains information about the most major fault in s22.

llama_perf_context_print:        load time =  107859.87 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   601 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    28 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  116660.83 ms /   629 tokens


 I don't know the answer to your question as none of the provided context contains information about the most major fault in s22.
